In [ ]:
########################################################################
######################## Carga de módulos ##############################
########################################################################

import numpy as np
import pandas as pd
from skimage.color import rgb2gray
from skimage.io import imread
from google.colab import files
import matplotlib.pyplot as plt
import os

In [ ]:
########################################################################
###################### Carga y visualización RGB #######################
########################################################################

# Subir archivo .jpg desde tu computadora
uploaded = files.upload()

# Leer imagen (usa la primera imagen que se cargue)
mi_imagen = list(uploaded.keys())[0]
img = imread(mi_imagen)

# Mostrar imagen RGB
plt.imshow(img)
plt.axis('off')
plt.title("mi imagen (RGB)")
plt.show()

# Mostrar las dimensiones
print("Dimensiones de la imagen:", img.shape)


In [ ]:
########################################################################
################### Conversión a escala de grises ######################
########################################################################

# Convertir la imagen RGB a escala de grises (matriz 2D)
imagen_gris = rgb2gray(img)

# Mostrar imagen en blanco y negro
plt.imshow(imagen_gris, cmap='gray')
plt.axis('off')
plt.title("Imagen en blanco y negro")
plt.show()

# Mostrar dimensiones
print("Dimensiones de la imagen en escala de grises:", imagen_gris.shape)
df = pd.DataFrame(imagen_gris)

# Guardamos en un archivo CSV
df.to_csv("imagen_gris.csv", index=False, header=False)

In [ ]:
########################################################################
################### Descomposición SVD de la imagen ####################
########################################################################

# Aplicar SVD
U, S, VT = np.linalg.svd(imagen_gris, full_matrices=False)

# Mostrar dimensiones
print("Dimensión de U:", U.shape)
print("Dimensión de S:", S.shape)   # Vector de valores singulares
print("Dimensión de VT:", VT.shape)

# Mostrar los primeros 10 valores singulares
print("\nPrimeros 10 valores singulares:")
print(S[:10])

In [ ]:
########################################################################
############ Reconstrucción de la imagen para varios r #################
########################################################################

rangos = [1, 5, 10, 20, 50, 100]

# Figura para visualización
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
fig.suptitle("Reconstrucciones SVD + Exportación CSV", fontsize=16)

# Reconstrucción, visualización y guardado
for ax, r in zip(axes.flat, rangos):
    # Reconstrucción con r valores singulares
    A_r = U[:, :r] @ np.diag(S[:r]) @ VT[:r, :]
    
    # Mostrar la imagen reconstruida
    ax.imshow(A_r, cmap='gray')
    ax.set_title(f"r = {r}")
    ax.axis('off')
    
    # Guardar como CSV
    df = pd.DataFrame(A_r)
    df.to_csv(f"mi_imagen_r{r}.csv", index=False)

plt.tight_layout()
plt.show()


In [ ]:
########################################################################
######## Energía acumulada con r óptimos marcados ######################
########################################################################

# Calcular energía acumulada
energia = np.cumsum(S**2) / np.sum(S**2)
r_vals = np.arange(1, len(energia) + 1)

# Buscar primeros r donde se supera cierto porcentaje
r_90 = np.argmax(energia >= 0.90) + 1
r_95 = np.argmax(energia >= 0.95) + 1
r_99 = np.argmax(energia >= 0.99) + 1

# Graficar
plt.figure(figsize=(10, 6))
plt.plot(r_vals, energia, label="Energía acumulada", linewidth=2)
plt.axhline(0.90, color='gray', linestyle='--', label="90%")
plt.axhline(0.95, color='orange', linestyle='--', label="95%")
plt.axhline(0.99, color='green', linestyle='--', label="99%")

# Marcar puntos clave
plt.plot(r_90, energia[r_90 - 1], 'o', color='gray')
plt.text(r_90 + 2, energia[r_90 - 1], f"r = {r_90}", color='gray')

plt.plot(r_95, energia[r_95 - 1], 'o', color='orange')
plt.text(r_95 + 2, energia[r_95 - 1], f"r = {r_95}", color='orange')

plt.plot(r_99, energia[r_99 - 1], 'o', color='green')
plt.text(r_99 + 2, energia[r_99 - 1], f"r = {r_99}", color='green')

# Etiquetas y estética
plt.title("Energía acumulada vs. número de componentes r")
plt.xlabel("r (número de valores singulares)")
plt.ylabel("Fracción de energía acumulada")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
########################################################################
### Reconstrucción y exportación CSV para r óptimos #######
########################################################################

# Valores óptimos
r_optimos = [1,4,41]

# Figura para visualización
fig, axes = plt.subplots(1, len(r_optimos), figsize=(15, 5))
fig.suptitle("Reconstrucciones óptimas según energía acumulada", fontsize=16)

# Reconstrucción y exportación
for ax, r in zip(axes, r_optimos):
    A_r = U[:, :r] @ np.diag(S[:r]) @ VT[:r, :]
    
    # Mostrar imagen
    ax.imshow(A_r, cmap="gray")
    ax.set_title(f"r = {r}")
    ax.axis("off")
    
    # Guardar CSV
    pd.DataFrame(A_r).to_csv(f"mi_imagen_opt_r{r}.csv", index=False)

plt.tight_layout()
plt.show()
